In [1]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,ReLU,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential
import tensorflow.keras.optimizers as optimizers

MODEL_LOSS = 'categorical_crossentropy'
MODEL_METRIC = 'categorical_accuracy'

def InitialiazeModel(lr):
    """
    head_only:选择是否只训练顶端（即自定义的全连接层）
    weights:选择是否从外部导入权重
    model:模型名称
    lr:学习率：默认为0.001
    """
    model = Sequential()
    in_shape=(512, 512, 3)
    model.add(Conv2D(64, (5, 5),kernel_initializer='he_uniform', strides=(3,3),padding='valid', input_shape=in_shape))
    model.add(BatchNormalization(momentum=0.99)) # 根据batch_size修改
    model.add(ReLU())
    model.add(MaxPooling2D((3, 3),strides=1,padding='same'))
    # repeat
    model.add(Conv2D(64, (5, 5),kernel_initializer='he_uniform', strides=(3,3),padding='valid'))
    model.add(BatchNormalization(momentum=0.99)) # 根据batch_size修改
    model.add(ReLU())
    model.add(MaxPooling2D((3, 3),strides=1,padding='same'))
    # repeat
    model.add(Conv2D(64, (5, 5),kernel_initializer='he_uniform', strides=(3,3),padding='valid'))
    model.add(BatchNormalization(momentum=0.99)) # 根据batch_size修改
    model.add(ReLU())
    model.add(MaxPooling2D((3, 3),strides=1,padding='same'))
    
    model.add(Flatten())
    model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    
    # ========================= 优化器 =======================================
    MODEL_OPTIMIZER = optimizers.SGD(lr=lr, momentum=0.95, nesterov=True)
    # MODEL_OPTIMIZER = optimizers.Adam()
    # 编译模型
    model.compile(loss=MODEL_LOSS, optimizer=MODEL_OPTIMIZER, metrics=[MODEL_METRIC])
    # model.summary()
    return model

In [2]:
InitialiazeModel(0.001)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 170, 170, 64)      4864      
_________________________________________________________________
batch_normalization (BatchNo (None, 170, 170, 64)      256       
_________________________________________________________________
re_lu (ReLU)                 (None, 170, 170, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 170, 170, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 64)        102464    
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 64)        256       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 56, 56, 64)        0

In [2]:
import os
category = os.listdir("datasets\\Train")
category.remove('.ipynb_checkpoints')
print(category)


['Apple_iPhone6Plus', 'Canon_PowerShotA640', 'Huawei_P9', 'Lenovo_P70A', 'Microsoft_Lumia640LTE', 'Nikon_D70s', 'OnePlus_A3003', 'Samsung_GalaxyS5', 'Sony_DSC-W170', 'Xiaomi_RedmiNote3']


第一次训练

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,ReduceLROnPlateau
from math import ceil

train_path = "datasets\\Train"
val_path = "datasets\\Val_main"
BATCH_SIZE = 32
train_sample_num = 30000
val_sample_num = 1555

In [4]:
# 训练集生成器
data_gen = ImageDataGenerator()
train_it = data_gen.flow_from_directory(directory=train_path,target_size=(512,512),
                                        classes=category,class_mode= "categorical",
                                        batch_size=BATCH_SIZE)
# 验证集生成器
data_gen = ImageDataGenerator()
val_it = data_gen.flow_from_directory(directory=val_path,target_size=(512,512),
                                        classes=category,class_mode= "categorical",
                                        batch_size=BATCH_SIZE)

Found 30150 images belonging to 10 classes.
Found 1555 images belonging to 10 classes.


In [4]:
# 训练
model = InitialiazeModel(lr=0.01)
weights_path_name = "model_weight\\alex\\alex{epoch:02d}.hdf5" 
callbacks = [ModelCheckpoint(weights_path_name, monitor='val_loss', save_best_only=True, verbose=0,
                                             save_weights_only=True),
             EarlyStopping(monitor='val_loss', patience=5, verbose=0.001),
             TensorBoard(log_dir='train_log\\Alex',update_freq='epoch'),
             ReduceLROnPlateau(factor=0.5,
                               patience=3, 
                              min_lr=0.001)]
history1 = model.fit_generator(generator = train_it,
                    validation_data = val_it,
                    epochs = 100,
                    steps_per_epoch=ceil(train_sample_num/ BATCH_SIZE),
                    validation_steps=ceil(val_sample_num/ BATCH_SIZE),
                   max_queue_size=20,
                   callbacks=callbacks,
                   verbose = 1)

Found 30150 images belonging to 10 classes.
Found 1555 images belonging to 10 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
  1/938 [..............................] - ETA: 0s - loss: 3.7022 - categorical_accuracy: 0.0625WARNING:tensorflow:From F:\anaconda\envs\tensorflow2.3.1\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
938/938 [==============================] - 238s 254ms/step - loss: 2.2788 - categorical_accuracy: 0.1648 - val_loss: 2.2316 - val_categorical_accuracy: 0.1961
Epoch 2/100
938/938 [==============================] - 262s 279ms/step - loss: 1.9771 - categorical_accuracy: 0.2664 - val_loss: 2.1838 - val_categorical_accuracy: 0.2244
Epoch 3/100
938/938 [==============================] - 266s 284ms/step - loss: 1.7326 - categorical_accuracy:

减小学习率继续

In [ ]:

# 减小学习率
model = InitialiazeModel(lr=0.001)
weights_path_name = "model_weight\\alex\\alex{epoch:02d}+24.hdf5"

#权重
weights = './model_weight/alex/alex19.hdf5'
model.load_weights(weights)
    
# 试试 adam
callbacks = [ModelCheckpoint(weights_path_name, monitor='val_loss', save_best_only=True, verbose=0,
                                             save_weights_only=True),
             EarlyStopping(monitor='val_loss', patience=5, verbose=0.001),
             TensorBoard(log_dir='train_log\\Alex',update_freq='epoch'),
             ReduceLROnPlateau(factor=0.5,
                               patience=2, 
                              min_lr=0.001)]

history2 = model.fit_generator(generator = train_it,
                    validation_data = val_it,
                    epochs = 100,
                    steps_per_epoch=ceil(train_sample_num/ BATCH_SIZE),
                    validation_steps=ceil(val_sample_num/ BATCH_SIZE),
                   max_queue_size=20,
                   callbacks=callbacks,
                   verbose = 1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
  1/938 [..............................] - ETA: 0s - loss: 0.1994 - categorical_accuracy: 0.9375WARNING:tensorflow:From F:\anaconda\envs\tensorflow2.3.1\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
938/938 [==============================] - 246s 262ms/step - loss: 0.2655 - categorical_accuracy: 0.9107 - val_loss: 0.8815 - val_categorical_accuracy: 0.7395
Epoch 2/100
938/938 [==============================] - 248s 264ms/step - loss: 0.2548 - categorical_accuracy: 0.9146 - val_loss: 0.8310 - val_categorical_accuracy: 0.7479
Epoch 3/100
938/938 [==============================] - 257s 274ms/step - loss: 0.2395 - categorical_accuracy: 0.9184 - val_loss: 0.9742 - val_categorical_accuracy: 0.7196
Epoch 4/100
938/938 [====